In [2]:
from util.scraper import scrap
from util.file_handler import create_directory_if_not_exists, make_file_path, saveAsJson, load_file_line_by_line
from util.driver_util import sleep_like_human

from selenium.webdriver.chrome.webdriver import WebDriver

from bs4 import BeautifulSoup
from bs4.element import Tag


In [3]:
from time import sleep
from random import uniform


In [4]:
from winsound import Beep

In [5]:
ROOT_DIR = make_file_path("saramin")
SOURCE_FILE = "saramin.txt"

create_directory_if_not_exists(ROOT_DIR)


In [6]:
def opationalGetter(instance, attr, default=None):
    return getattr(instance, attr) if instance else default

def opationalCall(instance, func, default=None):
    return getattr(instance, func) if instance else (lambda *args, **kargs: default)

def opationalAttr(instance, attr, default=None):
    return instance[attr] if instance else default


In [7]:
def optionalString(element):
    element : str = opationalGetter(element, "string")
    element : str = opationalCall(element, "strip")()
    return element

In [8]:
def name_parser(soup : BeautifulSoup):
    stages = [
        "body > div.company-wrap > div.company-header > div.company-header-container",
        "div.name"
    ]
    
    element = soup
    for stage in stages:
        element = element.select_one(stage)
    element : str = optionalString(element)
    return "기업명", element

In [9]:
def logo_parser(soup : BeautifulSoup):
    stages = [
        "body > div.company-wrap > div.company-header > div.company-header-container",
        "img.width"
    ]

    element = soup
    for stage in stages:
        element = element.select_one(stage)
    element : str = opationalAttr(element, "src")
    element : str = 'https:' + element if element else None
    return "로고_URL", element

In [10]:
def attr_parser(soup : BeautifulSoup):
    root : Tag = soup.select("#company-body > div.company-body-infomation > div.company-infomation-row.basic-infomation > div > table > tbody > tr.field")
    for el in root:
        arr = el.select("th.field-label, td.field-value")
        if len(arr) != 4: continue
        a,b,c,d = arr
        
        b = b.select_one("div.value, div.salary-average-item")
        d = d.select_one("div.value, div.salary-average-item")
        a,b,c,d = tuple(optionalString(i) for i in (a,b,c,d))
        
        yield a, b
        yield c, d

In [11]:
def html_to_json(soup : BeautifulSoup):
    result = {}
    
    for func in [name_parser, logo_parser]:
        key, val = func(soup)
        result[key] = val
    for key, val in attr_parser(soup):
        result[key] = val
    
    return result

In [13]:
def execute_plan_unit(driver : WebDriver, id : int):
    url = f"https://www.jobkorea.co.kr/company/{id}"
    driver.get(url)
    
    sleep_like_human()
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    return html_to_json(soup)

In [14]:
def execute_plan(driver : WebDriver, start : int = -1, end : int = 1000000000, black_list : tuple = tuple()):
    for idx, id in enumerate(load_file_line_by_line(SOURCE_FILE)):
        if idx < start: continue
        if idx >= end:  break
        if idx in black_list: continue
        
        print(idx, id)
        saveAsJson(f"{ROOT_DIR}/{id}.json", execute_plan_unit(driver, id))
        
        Beep(1000, 500)
        

In [17]:
scrap(lambda func: execute_plan(func, start=-1, end=200, black_list=(17,53,80,99,140,154)))

155 1977592
156 45962250
157 1637769
158 1544709
159 1595884
160 1774290
161 1552134
162 42960697
163 1659278
164 1880678
165 1686783
166 45673083
167 1563384
168 1688706
169 1906019
170 1802919
171 1976336
172 1392574
173 1523727
174 1833921
175 1791771
176 1913523
177 1650408
178 1363393
179 16151703
180 1528451
181 1891525
182 16153118
183 1620897
184 16152646
185 1831538
186 1394073
187 1636977
188 1938552
189 16152584
190 1407046
191 1947356
192 1554402
193 43042976
194 1568650
195 43501001
196 1601608
197 1756390
198 1447867
199 1820327
